## Win/Loss Rating Model Prediction

Load the model and make predictions

In [1]:
import requests
import pandas as pd
import numpy as np
import pymc3 as pm
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from scipy.stats import norm
from spcl_case import *
plt.style.use('fivethirtyeight')
%matplotlib inline

### Get a list of all CS Games

In [2]:
r = requests.get('https://thunderpick.com/api/matches')
games = pd.DataFrame(r.json()['data'])
games = games[games.gameId == 6].sort_values('championship')

bet_games = []
for i,v in games.iterrows():
    if((v['isTournament'] == False )& (v['canWager'] == True)):
        ratio = v['matchBet']['buckets'][0]['amount']/v['matchBet']['buckets'][1]['amount']
        odds = (ratio**-1+1, ratio+1)
        wr = (odds[1]/np.sum(odds)*100., odds[0]/np.sum(odds)*100.)
        det = requests.get('https://thunderpick.com/api/matches/'+str(v['id'])).json()['data']
        print('Date: %s  |  Event: %s  | (BO%s) %s vs. %s  |  (%.1f:%.1f) | Total Coins: %i' % 
              (v['startTime'][:10], v['championship'], det['bestOfMaps'], v['matchBet']['buckets'][0]['label'], 
               v['matchBet']['buckets'][1]['label'], wr[0], wr[1], v['matchBet']['amount']))
        bet_games.append({'1': v['matchBet']['buckets'][0]['label'], '2': v['matchBet']['buckets'][1]['label'], 'bo': det['bestOfMaps'], 'o1': odds[0], 'o2': odds[1], 'wr': wr[0]})
bet_games = pd.DataFrame(bet_games)

Date: 2017-10-26  |  Event: ECS Season 4 Development League  | (BO1) Ghost vs. Immortals  |  (37.0:63.0) | Total Coins: 2000
Date: 2017-10-25  |  Event: ECS Season 4 Development League  | (BO1) EnVyUs vs. fnatic  |  (52.0:48.0) | Total Coins: 2000
Date: 2017-10-25  |  Event: ECS Season 4 Development League  | (BO1) fnatic vs. EnVyUs  |  (45.0:55.0) | Total Coins: 2000
Date: 2017-10-25  |  Event: ECS Season 4 Development League  | (BO1) FaZe vs. mousesports  |  (63.2:36.8) | Total Coins: 10324
Date: 2017-10-25  |  Event: ECS Season 4 Development League  | (BO1) mousesports vs. FaZe  |  (36.1:63.9) | Total Coins: 2050
Date: 2017-10-24  |  Event: ECS Season 4 Development League  | (BO1) G2 vs. Heroic  |  (68.5:31.5) | Total Coins: 2540
Date: 2017-10-26  |  Event: ECS Season 4 Development League  | (BO1) Liquid vs. SK  |  (40.0:60.0) | Total Coins: 10000
Date: 2017-10-26  |  Event: ECS Season 4 Development League  | (BO1) SK vs. Liquid  |  (62.8:37.2) | Total Coins: 2150
Date: 2017-10-25  

## Load Ratings Model

In [42]:
TEAM_SET = 'mdlau'

teams = np.load('saved_model/'+TEAM_SET+'/teams.npy')
maps = np.load('saved_model/'+TEAM_SET+'/maps.npy')
filt = np.load('saved_model/'+TEAM_SET+'/filter_teams.npy')
h_teams = pd.read_csv('hltv_csv/teams.csv').set_index('ID').loc[teams]
h_teams = fix_teams(h_teams)

h_teams_filt = h_teams[h_teams.Name.isin(filt)]

def prep_pymc_model(n_teams, n_maps):
    with pm.Model() as rating_model:
        omega = pm.HalfCauchy('omega', 0.5)
        tau = pm.HalfCauchy('tau', 0.5)
        rating = pm.Normal('rating', 0, omega, shape=n_teams)
        theta_tilde = pm.Normal('rate_t', mu=0, sd=1, shape=(n_maps, n_teams))
        rating_map = pm.Deterministic('rating | map', rating + tau * theta_tilde)
        alpha = pm.Gamma('alpha', 10, 5)
        sigma = pm.HalfCauchy('sigma', 0.5, shape=n_maps)
    return rating_model

rating_model = prep_pymc_model(len(teams), len(maps))
trace = pm.backends.text.load('saved_model/'+TEAM_SET+'/trace', model=rating_model)

## Ban/Pick Predictions

In [43]:
h_bp = pd.read_csv('hltv_csv/picksAndBans.csv').set_index('Match ID')
h_matches = pd.read_csv('hltv_csv/matchResults.csv').set_index('Match ID')
h_matches['Date'] = pd.to_datetime(h_matches['Date'])
h_matches = h_matches[h_matches['Date'] >= dt.datetime(2017,1,1)]
h_bp = h_bp.join(h_matches[['Date']], how='left')
h_bp['Date'] = pd.to_datetime(h_bp['Date'])
h_bp = h_bp[h_bp['Date'] >= dt.datetime(2017,1,1)]

In [44]:
def model_mp(train, t1, t2):
    tab = train[train['Team'].isin([t1, t2])].groupby(['Team', ' Pick Type', 'Map'])['Date'].count().unstack([' Pick Type', 'Team']).fillna(0)
    return (tab/tab.sum(axis=0)).mean(level=0,axis=1)# get average

def model_played(train, t1, t2):
    a = train[train['Team 1 ID'].isin([t1,t2])].groupby(['Team 1 ID', 'Map'])['Date'].count()
    b = train[train['Team 2 ID'].isin([t1,t2])].groupby(['Team 2 ID', 'Map'])['Date'].count()
    c = pd.DataFrame([a,b], index=['a','b']).T.fillna(0)
    c = (c['a']+c['b']).unstack(level=0).fillna(0)
    return (c/c.sum()).mean(axis=1)

def predict_map(func, data, t1, t2):
    res = func(data, t1, t2)
    return res.loc[res.index != 'Default'].sort_values(ascending=False)

# Bet Predictions

In [50]:
money = 4500.
bet_games['1'] = bet_games['1'].str.replace('ex-Denial', 'Denial')
bet_games['2'] = bet_games['2'].str.replace('ex-Denial', 'Denial')
matches = bet_games[bet_games['1'].isin(filt) & bet_games['2'].isin(filt)].drop_duplicates()
def sig(x):
    return 1 / (1 + np.exp(-x))
def abs_norm_interval(start,end,loc,scale):
    return (norm.cdf(end,loc,scale) - norm.cdf(start,loc,scale)) + (norm.cdf(-1*start,loc,scale) - norm.cdf(-1*end,loc,scale))

t_rating = trace['rating']
t_map_rating = trace['rating | map']
t_alpha = trace['alpha']
for i,v in matches.iterrows():
    t1_id = h_teams_filt[h_teams_filt.Name == v['1']].index[0]; t1_ind = np.where(teams == t1_id)[0][0];
    t2_id = h_teams_filt[h_teams_filt.Name == v['2']].index[0]; t2_ind = np.where(teams == t2_id)[0][0];
    trace_1 = t_rating[:,t1_ind]; trace_2 = t_rating[:,t2_ind]
    mr_1 = trace_1.mean(); mr_2 = trace_2.mean();
    diff = trace_1-trace_2
    p_wl = sig(diff)
    wr_25 = np.percentile(p_wl, 25); wr_75 = np.percentile(p_wl, 75)
    kelly_pct_1 = ((v['o1']*np.percentile(p_wl, 45)-(1.-np.percentile(p_wl, 45)))/v['o1'])*0.1
    kelly_pct_2 = ((v['o2']*(1.-np.percentile(p_wl, 45))-(np.percentile(p_wl, 45)))/v['o2'])*0.1
    print('%s (%.3f) vs %s (%.3f) - I:%.2f%% | P:%.2f%% - %.2f%%  -  K: %.1f%% (%i) - %.1f%% (%i)' % 
          (v['1'], mr_1, v['2'], mr_2, v['wr'], wr_25*100, wr_75*100, kelly_pct_1*100., 
           kelly_pct_1*money, kelly_pct_2*100., kelly_pct_2*money))

SYF (-0.139) vs Corvidae (-0.671) - I:43.37% | P:52.53% - 72.13%  -  K: 4.4% (197) - 0.5% (20)
SYF (-0.139) vs Athletico (0.814) - I:40.00% | P:20.30% - 36.69%  -  K: -0.3% (-12) - 5.7% (258)
Kings (2.662) vs Chiefs (2.253) - I:60.49% | P:50.79% - 68.58%  -  K: 3.3% (150) - 1.8% (82)
Grayhound (2.244) vs Legacy (0.913) - I:60.00% | P:72.60% - 84.33%  -  K: 6.5% (291) - -0.9% (-41)
Athletico (0.814) vs seadoggs (1.028) - I:67.00% | P:31.06% - 59.26%  -  K: 0.3% (15) - 4.4% (197)
Athletico (0.814) vs Kings (2.662) - I:40.00% | P:9.75% - 18.87%  -  K: -2.2% (-98) - 7.9% (357)


In [53]:
PRINT_RD_DIFF = False
for i,v in matches.iterrows():
    t1_id = h_teams_filt[h_teams_filt.Name == v['1']].index[0]; t1_ind = np.where(teams == t1_id)[0][0];
    t2_id = h_teams_filt[h_teams_filt.Name == v['2']].index[0]; t2_ind = np.where(teams == t2_id)[0][0];
    pred_maps = predict_map(model_played, h_matches, t1_id, t2_id)
    pred_maps = pred_maps/pred_maps.sum()
    for m,s in pred_maps.iteritems():
        m_ind = np.where(maps == m)[0][0]
        trace_1 = t_map_rating[:,m_ind,t1_ind]; trace_2 = t_map_rating[:,m_ind,t2_ind]
        mr_1 = trace_1.mean(); mr_2 = trace_2.mean();
        diff = trace_1-trace_2
        p_wl = sig(diff)
        wr_25 = np.percentile(p_wl, 25); wr_75 = np.percentile(p_wl, 75)
        kappa = 32*sig(t_alpha*diff)-16.
        kelly_pct_1 = ((v['o1']*np.percentile(p_wl, 45)-(1.-np.percentile(p_wl, 45)))/v['o1'])*0.1
        kelly_pct_2 = ((v['o2']*(1.-np.percentile(p_wl, 45))-(np.percentile(p_wl, 45)))/v['o2'])*0.1
        print('    Map: %s (%.2f)  -  %s (%.3f) vs %s (%.3f) - I:%.2f%% | P:%.2f%% - %.2f%%  -  K: %.1f%% (%i) - %.1f%% (%i)' % 
             (m, s*100., v['1'], mr_1, v['2'], mr_2, v['wr'], wr_25*100, wr_75*100, kelly_pct_1*100., 
               kelly_pct_1*money, kelly_pct_2*100., kelly_pct_2*money))
        
        if(PRINT_RD_DIFF):
            p_sc = [abs_norm_interval(x[0],x[1],kappa,trace['sigma'][:,m_ind]) for x in [[1.5,3.5],[3.5,5.5],[5.5,7.5],[7.5,9.5],[9.5,16]]]
            for i,sd in enumerate(['2 - 3 Rounds', '4 - 5 rounds', '6 - 7 rounds', '8 - 9 rounds', '10 rounds or more']):
                sc_25 = np.percentile(p_sc[i], 25); sc_75 = np.percentile(p_sc[i], 75)
                print('      %s : %.2f%% - %.2f%%' % (sd, sc_25*100, sc_75*100))

    Map: Mirage (27.55)  -  SYF (0.514) vs Corvidae (-0.510) - I:43.37% | P:65.63% - 80.33%  -  K: 6.0% (268) - -1.3% (-56)
    Map: Cobblestone (19.81)  -  SYF (1.618) vs Corvidae (-0.734) - I:43.37% | P:86.21% - 94.43%  -  K: 8.6% (388) - -4.2% (-187)
    Map: Cache (14.37)  -  SYF (-1.279) vs Corvidae (-0.379) - I:43.37% | P:18.83% - 41.54%  -  K: -0.5% (-20) - 5.8% (259)
    Map: Train (12.51)  -  SYF (-0.563) vs Corvidae (-0.672) - I:43.37% | P:38.23% - 67.06%  -  K: 2.8% (126) - 2.2% (98)
    Map: Overpass (12.29)  -  SYF (0.159) vs Corvidae (0.047) - I:43.37% | P:37.80% - 67.19%  -  K: 2.9% (129) - 2.1% (95)
    Map: Inferno (8.25)  -  SYF (-0.461) vs Corvidae (-0.795) - I:43.37% | P:42.57% - 72.55%  -  K: 3.6% (161) - 1.3% (60)
    Map: Nuke (5.22)  -  SYF (-0.997) vs Corvidae (-1.789) - I:43.37% | P:50.51% - 82.70%  -  K: 5.1% (227) - -0.3% (-12)
    Map: Mirage (24.55)  -  SYF (0.514) vs Athletico (1.177) - I:40.00% | P:26.23% - 43.19%  -  K: 0.6% (25) - 4.8% (215)
    Map: C

In [34]:
m

'Default'

In [ ]:
plt.ylim(0,1.2)
sns.kdeplot(trace_1, shade=True, alpha=0.65, legend=True, label=v['1'])
sns.kdeplot(trace_2, shade=True, alpha=0.65, legend=True, label=v['2'])

In [ ]:
h_bp.groupby('Match ID').first().count()

In [ ]:
h_bp